In [1]:
import numpy as np
import pandas as pd
from math import log

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [3]:
class NoArvore:
    def __init__(self, indices_treino, profundidade=None):
        self.filho_direito = None
        self.filho_esquerdo = None
        self.indices_treino = indices_treino
        self.indices_predicao = None
        self.predicao = 0
        self.entropia = 0
        self.profundidade = profundidade
        self.ganho_informacao = 0
        self.feature = None
        self.corte = None
        self.folha = False
        self.quantidade_dados = (self.indices_treino==True).shape[0]

In [4]:
class ArvoreClassificacaoSimples():
    def __init__(self):
        self.dados = None
        self.colunas_preditoras = None
        self.coluna_resposta = None
        self.num_features = None
        self.numero_classes = None
        self.raiz = None
        self.fila_nos = list()
        
    def __entropia(self, node):
        h = 0

        for k in range(self.numero_classes):
            pk = node.predicao[k]

            # nao queremos log(0)
            if pk != 0:
                h += pk * log(pk,2)

        return -h
    
    def __entropia_divisao(self, no_esquerdo, no_direito):
        respostas_esquerda = self.dados.iloc[no_esquerdo.indices_treino][self.coluna_resposta]
        respostas_direita = self.dados.iloc[no_direito.indices_treino][self.coluna_resposta]
        
        n_esquerda = respostas_esquerda.shape[0]
        n_direita = respostas_direita.shape[0]
        total_repostas = n_esquerda + n_direita

        h = ((float(n_esquerda) / total_repostas) * no_esquerdo.entropia + \
              (float(n_direita) / total_repostas) * no_direito.entropia)
        
        return h

    
    def __predicao_no(self, indices_treino):
        respostas_no = self.dados.iloc[indices_treino][self.coluna_resposta]
        n = respostas_no.shape[0]
       
        # proporcao para casa uma das classes
        pred = []
        for k in range(self.numero_classes):
            nk = (respostas_no == k).sum()
            pk = float(nk) / n
            pred.append(pk)

        return pred
    
    def __cria_no(self, indices_treino, profundidade=0):
        node = NoArvore(indices_treino, profundidade)

        node.predicao = self.__predicao_no(indices_treino)
        node.entropia = self.__entropia(node)

        return node
    
    def __divide_no(self, node):
        dados_no = self.dados.iloc[node.indices_treino]
        respostas_no = dados_no[self.coluna_resposta]
        num_amostras = node.indices_treino.size
        
        # se for no puro ou tiver menos que 2 amostras
        if respostas_no.nunique() == 1 or num_amostras < 2:
            node.folha = True
            return

        melhor_candidato = None
        maior_ganho = np.inf
        
        # para todas as features
        for f in self.colunas_preditoras:
            col = dados_no[f]

            valores_unicos = np.sort(np.unique(col))
            num_valores = len(valores_unicos)

            # para todos possiveis valores da feature
            for i in range(0, num_valores - 1):
                t = (valores_unicos[i] + valores_unicos[i+1]) / 2.
                indice_esquerdo = node.indices_treino[(col < t)]
                indice_direito = node.indices_treino[(col >= t)]
                
                # nao considera splits que deixam um dos filhos sem amostra
                if indice_esquerdo.size == 0 or indice_direito.size == 0:
                    continue
                    
                no_esquerdo = self.__cria_no(
                                    indice_esquerdo,
                                    node.profundidade + 1)
                no_direito = self.__cria_no(
                                    indice_direito,
                                    node.profundidade + 1)

                entropia_div = self.__entropia_divisao(no_esquerdo, no_direito)
                if entropia_div < maior_ganho:
                    maior_ganho = entropia_div
                    melhor_candidato = [f, t, entropia_div, no_esquerdo, no_direito]


        if melhor_candidato:
            ganho_info = node.entropia - melhor_candidato[2]

            node.feature = melhor_candidato[0]
            node.corte = melhor_candidato[1]
            node.filho_esquerdo = melhor_candidato[3]
            node.filho_direito = melhor_candidato[4]
            node.ganho_informacao = ganho_info

            print(node.feature)
            print(node.corte)
            print(node.ganho_informacao)
            
            # adiciona os novos nos para serem explorados
            self.fila_nos.append(node.filho_esquerdo)
            self.fila_nos.append(node.filho_direito)


    def treino(self, dados, colunas_preditoras, coluna_resposta):
        # dados: DataFrame pandas
        # colunas_preditora: lista com strings dos nomes das colunas das variáveis de predição (features)
        # coluna_resposta: string do nome da coluna da variável de resposta
        #
        # atualize o valor de self.dados, self.coluna_preditora e 
        # self.coluna_resposta
        #
        #
        # OBS: nenhuma das outras funções deve ser rodada antes do treino.
        self.dados = dados
        self.colunas_preditoras = colunas_preditoras
        self.coluna_resposta = coluna_resposta
        
        self.tamanho_treino = dados.shape[0]
        self.numero_classes = dados[coluna_resposta].nunique()

        indice_inicial = np.arange((self.tamanho_treino))

        raiz = self.__cria_no(indice_inicial)
        self.fila_nos.append(raiz)
        self.raiz = raiz

        while self.fila_nos:
            node = self.fila_nos.pop(0)
            self.__divide_no(node)

        return 

    def predicao(self, x):
        # x: DataFrame pandas com as mesmas colunas preditoras
        #
        # implemente e retorne a predicao usando a arvore aprendida no treino
        
        tamanho_dados = x.shape[0]
        predicoes = np.zeros((tamanho_dados), dtype=float)

        indices_dados = np.arange((tamanho_dados))
        
        self.raiz.indices_predicao = indices_dados
        nos = [self.raiz]

        while nos:
            node = nos.pop(0)

            # no folha
            if node.folha:
                k = np.argmax(node.predicao)
                predicoes[node.indices_predicao] = k

                continue
            
            col = x.iloc[node.indices_predicao][node.feature]
            indices_esquerda = node.indices_predicao[(col < node.corte)]
            indices_direita = node.indices_predicao[(col >= node.corte)]
            
            node.filho_esquerdo.indices_predicao = indices_esquerda
            node.filho_direito.indices_predicao = indices_direita

            nos.append(node.filho_esquerdo)
            nos.append(node.filho_direito)

        return predicoes


In [5]:
from sklearn.datasets import load_iris

iris = load_iris()

In [6]:
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                           columns= iris['feature_names'] + ['target'])

In [7]:
treino, teste = train_test_split(iris_df, random_state=12, test_size=0.2)

In [8]:
minha_dt = ArvoreClassificacaoSimples()

In [9]:
minha_dt.treino(treino, ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'], 'target')

petal length (cm)
2.45
0.9007196798623591
petal length (cm)
4.85
0.7188062035679573
petal width (cm)
1.65
0.21400141833049466
petal width (cm)
1.7000000000000002
0.14680904022894664
sepal width (cm)
3.1
0.9182958340544896
sepal width (cm)
3.05
0.31668908831502096
sepal length (cm)
6.05
0.3219280948873623
sepal width (cm)
2.45
1.0


In [10]:
pred = minha_dt.predicao(teste[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']])

In [11]:
y_teste = teste["target"].values

In [12]:
accuracy_score(y_teste, pred)

0.9333333333333333